## Import

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
print(os.environ.get('CUDA_VISIBLE_DEVICES'))

2


In [2]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# CUDA 장치 사용 가능 여부 확인
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


## Utils

In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

## Data Loader

In [5]:
transform = A.Compose(
    [   
        #A.Resize(224, 224),
        A.Resize(256, 256),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

## Define Model

In [6]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 13, 1) # 12개 class + 1 background

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   

        x = self.dconv_down4(x)

        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

## Model Train

In [7]:
# # model 초기화
# model = UNet().to(device)

# # loss function과 optimizer 정의
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # training loop
# for epoch in tqdm(range(20)):  # 20 에폭 동안 학습합니다.
#     model.train()
#     epoch_loss = 0
#     for images, masks in tqdm(dataloader):
#         images = images.float().to(device)
#         masks = masks.long().to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, masks.squeeze(1))
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

#     print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')

# Validation

In [8]:
# valid_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform, infer=True)
# valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)

# # 클래스(레이블) 수
# num_classes = 12

# # 클래스별 IoU를 계산하기 위한 함수
# def calculate_iou_per_class(y_true, y_pred, class_id):
#     intersection = np.sum((y_true == class_id) & (y_pred == class_id))
#     union = np.sum((y_true == class_id) | (y_pred == class_id))
#     iou = intersection / union if union > 0 else 0
#     return iou

# with torch.no_grad():
#     model.eval()
#     result = []
#     valid_true = []
#     for images, masks in tqdm(valid_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # 클래스별 IoU를 계산하고 기록
#         class_ious = []
#         for class_id in range(num_classes):
#             iou = calculate_iou_per_class(masks, np.array(outputs), class_id)
#             class_ious.append(iou)
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
#         # Validation 데이터셋의 정답 이미지를 로드하여 mIoU 계산


# # 클래스별 IoU 출력
# for class_id, iou in enumerate(class_ious):
#     print(f'Class {class_id} IoU: {iou:.4f}')

# # mIoU 계산
# mIoU = np.mean(class_ious)
# print(f"mIoU Score: {mIoU:.4f}")

In [9]:
import wandb
import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="Design_PJ",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.005,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 15,
#     }
# )
# model 초기화
model = UNet().to(device)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# 클래스(레이블) 수
num_classes = 13

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou


# training loop
for epoch in range(5):  # 5 에폭 동안 학습합니다.
          
    # 클래스별 IoU를 누적할 리스트 초기화
    class_ious = []
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # validation 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(num_classes):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            class_ious.append(iou)
            train_class_ious.append(iou)

    # mIoU 계산
    train_mIoU = np.mean(class_ious)
    train_class_ious = np.array(train_class_ious).reshape(num_classes, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    #mIoU = np.mean(class_ious)
    #print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(dataloader)}, train mIoU Score: {mIoU:.4f}")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')

    # validation
    val_loss = 0
    class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)
            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(num_classes):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                class_ious.append(iou)
                val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(num_classes, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')

                
    # for class_id, iou in enumerate(class_ious):
    # print(f'Class {class_id} IoU: {iou:.4f}')           
   
    # mIoU 계산
    val_mIoU = np.mean(class_ious)

#     # log metrics to wandb
#     wandb.log({"Loss1": (epoch_loss/len(dataloader))})
#     wandb.log({"Train Score": train_mIoU})
#     wandb.log({"Loss2": (val_loss/len(valid_dataloader))})
#     wandb.log({"Validation Score": val_mIoU})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

100%|██████████| 69/69 [02:19<00:00,  2.02s/it]


Class 0 IoU: 0.0158
Class 1 IoU: 0.0161
Class 2 IoU: 0.0127
Class 3 IoU: 0.0217
Class 4 IoU: 0.0121
Class 5 IoU: 0.0231
Class 6 IoU: 0.0276
Class 7 IoU: 0.0171
Class 8 IoU: 0.0190
Class 9 IoU: 0.0173
Class 10 IoU: 0.0168
Class 11 IoU: 0.0207
Class 12 IoU: 0.0219


100%|██████████| 15/15 [00:28<00:00,  1.88s/it]


Class 0 IoU: 0.0278
Class 1 IoU: 0.0311
Class 2 IoU: 0.0285
Class 3 IoU: 0.0278
Class 4 IoU: 0.0454
Class 5 IoU: 0.0281
Class 6 IoU: 0.0395
Class 7 IoU: 0.0296
Class 8 IoU: 0.0239
Class 9 IoU: 0.0296
Class 10 IoU: 0.0469
Class 11 IoU: 0.0275
Class 12 IoU: 0.0339


100%|██████████| 69/69 [02:19<00:00,  2.02s/it]


...

Class 0 IoU: 0.0523
Class 1 IoU: 0.0525
Class 2 IoU: 0.0589
Class 3 IoU: 0.0678
Class 4 IoU: 0.0534
Class 5 IoU: 0.0583
Class 6 IoU: 0.0634
Class 7 IoU: 0.0582
Class 8 IoU: 0.0579
Class 9 IoU: 0.0652
Class 10 IoU: 0.0601
Class 11 IoU: 0.0626
Class 12 IoU: 0.0695


100%|██████████| 15/15 [00:32<00:00,  2.20s/it]


Class 0 IoU: 0.0577
Class 1 IoU: 0.0619
Class 2 IoU: 0.0582
Class 3 IoU: 0.0578
Class 4 IoU: 0.0761
Class 5 IoU: 0.0572
Class 6 IoU: 0.0968
Class 7 IoU: 0.0602
Class 8 IoU: 0.0536
Class 9 IoU: 0.0597
Class 10 IoU: 0.0783
Class 11 IoU: 0.0572
Class 12 IoU: 0.0905


 12%|█▏        | 8/69 [00:24<03:09,  3.10s/it]


KeyboardInterrupt: 

## Inference

In [ ]:
# #test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:

# # 클래스(레이블) 수
# num_classes = 12

# # training loop
# for epoch in range(1):  # 5 에폭 동안 학습합니다.
          
#     # 클래스별 IoU를 누적할 리스트 초기화
#     class_ious = []
    
#     # validation
#     val_loss = 0
#     class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
#     with torch.no_grad():
#         model.eval()
#         for images, masks in tqdm(valid_dataloader):
#             images = images.float().to(device)
#             masks = masks.long().to(device)
#             outputs = model(images)

#             # validation loss 계산
#             val_loss += criterion(outputs, masks.squeeze(1)).item()

#             # validation 클래스별 IoU 계산
#             outputs = torch.softmax(outputs, dim=1).cpu()
#             outputs = torch.argmax(outputs, dim=1).numpy()

#             for class_id in range(num_classes):
#                 iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
#                 class_ious.append(iou)
                
#     for class_id, iou in enumerate(class_ious):
#         print(f'Class {class_id} IoU: {iou:.4f}')           
   
#     # mIoU 계산
#     val_mIoU = np.mean(class_ious)



In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
    


  0%|          | 0/30 [00:03<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (2,224,224) (16,224,224) 

## Submission

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result
# submit

,id,mask_rle
0,TEST_0000_class_0,477314 4 478274 4 479242 5 479255 9 480202 5 4...
1,TEST_0000_class_1,-1
2,TEST_0000_class_2,1 167 601 527 1561 531 2521 531 3481 531 4441 ...
3,TEST_0000_class_3,462824 13 463784 13 464744 13 465704 17 466664...
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,944 17 1904 17 2860 4 3820 4 4780 4 5757 4 671...
22772,TEST_1897_class_8,112 528 682 120 1072 528 1642 120 2037 518 260...
22773,TEST_1897_class_9,209002 9 209962 9 220021 4 220981 4 221937 4 2...
22774,TEST_1897_class_10,-1


In [ ]:
# submit.to_csv('C:\Workspace\Design_project\submission/baseline_submit.csv', index=False)